In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!cd /content
!rm -rf slm-hosting-playbook || true
!git clone https://github.com/bh3r1th/slm-hosting-playbook.git slm-hosting-playbook
!cd slm-hosting-playbook


Cloning into 'slm-hosting-playbook'...
remote: Enumerating objects: 314, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 314 (delta 81), reused 161 (delta 73), pack-reused 142 (from 2)
Receiving objects: 100% (314/314), 6.81 MiB | 19.88 MiB/s, done.
Resolving deltas: 100% (96/96), done.


In [4]:
%%bash
set -euo pipefail

cd /content/slm-hosting-playbook/gemma-slm-hosting

cp -f .env.example .env

# (Optional) override adapter path here if needed:
# ADAPTER_PATH="/content/drive/MyDrive/...."
# sed -i "s|^ADAPTER_PATH=.*|ADAPTER_PATH=${ADAPTER_PATH}|g" .env

# Quick sanity checks (read ADAPTER_PATH from .env)
ADAPTER_PATH="$(grep -E '^ADAPTER_PATH=' .env | head -n 1 | cut -d= -f2-)"
test -d "$ADAPTER_PATH"
test -f "$ADAPTER_PATH/adapter_config.json"
test -f "$ADAPTER_PATH/adapter_model.safetensors"

echo "✅ .env ready at gemma-slm-hosting/.env"
echo "Adapter path: $ADAPTER_PATH"


✅ .env ready at gemma-slm-hosting/.env
Adapter path: /content/drive/MyDrive/slm-hosting-playbook-artifacts/live_backup/gemma-3-1b-it-lora-20260114-034307


In [5]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

# Ensure BASE_API_URL / FT_API_URL include /v1
if grep -qE '^BASE_API_URL=http://127\.0\.0\.1:8000$' .env; then
  sed -i 's|^BASE_API_URL=http://127\.0\.0\.1:8000$|BASE_API_URL=http://127.0.0.1:8000/v1|g' .env
fi
if grep -qE '^FT_API_URL=http://127\.0\.0\.1:8001$' .env; then
  sed -i 's|^FT_API_URL=http://127\.0\.0\.1:8001$|FT_API_URL=http://127.0.0.1:8001/v1|g' .env
fi

# OPTIONAL: Colab VRAM safety (uncomment if you see OOM)
# sed -i 's|^MAX_NUM_SEQS=.*|MAX_NUM_SEQS=8|g' .env
# sed -i 's|^GPU_MEMORY_UTILIZATION=.*|GPU_MEMORY_UTILIZATION=0.85|g' .env

echo "== Effective endpoints =="
grep -E '^(BASE_API_URL|FT_API_URL|MAX_NUM_SEQS|GPU_MEMORY_UTILIZATION)=' .env || true


== Effective endpoints ==
BASE_API_URL=http://127.0.0.1:8000/v1
FT_API_URL=http://127.0.0.1:8001/v1
GPU_MEMORY_UTILIZATION=0.90
MAX_NUM_SEQS=128


In [6]:
%%bash
set -euo pipefail

python -m pip install -U pip
python -m pip install -U "vllm"

python -c "import vllm; print('vLLM:', vllm.__version__)"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.9/474.9 MB 48.5 MB/s  0:00:06
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 109.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 119.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 80.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 124.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 107.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 97.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 72.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.6/821.6 kB 44.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [7]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

make setup


mkdir -p runs/logs runs/pids runs/ab runs/perf runs/diag
GPU: NVIDIA A100-SXM4-40GB
VRAM: 40960 MiB
python -m pip install -r requirements.txt
python -c "import sys; print(sys.version.split()[0])"
3.12.12
python -c "import torch; print(torch.__version__)"
2.9.0+cu126
python -c "import vllm; print(vllm.__version__)"
0.13.0


In [8]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting
make start-both


bash scripts/start_base_vllm.sh
Expected endpoints:
http://0.0.0.0:8000/v1/models
http://0.0.0.0:8000/v1/chat/completions
Started base vLLM (pid 1363)
Logs: /content/slm-hosting-playbook/gemma-slm-hosting/runs/logs/base.log
bash scripts/start_ft_vllm.sh
Starting FT vLLM
HOST=0.0.0.0
PORT=8001
BASE_MODEL_ID=google/gemma-3-1b-it
ADAPTER_PATH=/content/drive/MyDrive/slm-hosting-playbook-artifacts/live_backup/gemma-3-1b-it-lora-20260114-034307
Expected endpoints:
http://0.0.0.0:8001/v1/models
http://0.0.0.0:8001/v1/chat/completions
Started FT vLLM (pid 1376)
Logs: /content/slm-hosting-playbook/gemma-slm-hosting/runs/logs/ft.log
Started base and ft. PIDs in runs/pids/


In [9]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

mkdir -p data
cat > data/perf_prompts.jsonl <<'EOF'
{"id":"p1","messages":[{"role":"user","content":"Say hello in one sentence."}]}
{"id":"p2","messages":[{"role":"user","content":"Explain LoRA in 2 short sentences."}]}
{"id":"p3","messages":[{"role":"user","content":"Give 3 bullets on why GPU is needed for LLM inference."}]}
{"id":"p4","messages":[{"role":"user","content":"Write a short Python function that adds two numbers."}]}
{"id":"p5","messages":[{"role":"user","content":"Summarize vLLM in one sentence."}]}
EOF

wc -l data/perf_prompts.jsonl
head -n 2 data/perf_prompts.jsonl


5 data/perf_prompts.jsonl
{"id":"p1","messages":[{"role":"user","content":"Say hello in one sentence."}]}
{"id":"p2","messages":[{"role":"user","content":"Explain LoRA in 2 short sentences."}]}


In [10]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

echo "== prompts candidates =="
find . -maxdepth 3 -type f \( -name "*prompts*.jsonl" -o -name "*prompts*.txt" -o -name "*prompts*.json" \) -print || true

echo
echo "== list data/ if exists =="
ls -la data 2>/dev/null || true

echo
echo "== smoke_test endpoints it will call (sanity) =="
python -c "import os; print('BASE_API_URL=', os.getenv('BASE_API_URL')); print('FT_API_URL=', os.getenv('FT_API_URL'))" || true


== prompts candidates ==
./loadtest/workloads/prompts_long.jsonl
./loadtest/workloads/prompts_short.jsonl
./data/prompts.jsonl
./data/perf_prompts.jsonl

== list data/ if exists ==
total 16
drwxr-xr-x  2 root root 4096 Jan 18 03:06 .
drwxr-xr-x 16 root root 4096 Jan 18 03:07 ..
-rw-r--r--  1 root root  467 Jan 18 03:08 perf_prompts.jsonl
-rw-r--r--  1 root root 1218 Jan 18 03:06 prompts.jsonl

== smoke_test endpoints it will call (sanity) ==
BASE_API_URL= None
FT_API_URL= None


In [11]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

mkdir -p data
ln -sf ../loadtest/workloads/prompts_short.jsonl data/prompts.jsonl

ls -la data
head -n 2 data/prompts.jsonl


total 12
drwxr-xr-x  2 root root 4096 Jan 18 03:08 .
drwxr-xr-x 16 root root 4096 Jan 18 03:07 ..
-rw-r--r--  1 root root  467 Jan 18 03:08 perf_prompts.jsonl
lrwxrwxrwx  1 root root   41 Jan 18 03:08 prompts.jsonl -> ../loadtest/workloads/prompts_short.jsonl
{"prompt": "Say hello in one sentence."}
{"prompt": "List three colors."}


In [13]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

# recreate .env from the repo's example
cp -f .env.example .env

# sanity
grep -E '^(HOST|BASE_PORT|FT_PORT|BASE_MODEL_ID|ADAPTER_PATH)=' .env


BASE_MODEL_ID=google/gemma-3-1b-it
ADAPTER_PATH=/content/drive/MyDrive/slm-hosting-playbook-artifacts/live_backup/gemma-3-1b-it-lora-20260114-034307
HOST=0.0.0.0
BASE_PORT=8000
FT_PORT=8001


In [14]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

make stop >/dev/null 2>&1 || true
make start-base
sleep 10
python scripts/smoke_test.py --mode base


bash scripts/start_base_vllm.sh
Expected endpoints:
http://0.0.0.0:8000/v1/models
http://0.0.0.0:8000/v1/chat/completions
Started base vLLM (pid 2289)
Logs: /content/slm-hosting-playbook/gemma-slm-hosting/runs/logs/base.log
MODE=base
models_url=http://127.0.0.1:8000/v1/models
chat_url=http://127.0.0.1:8000/v1/chat/completions
chat_method=POST
chat_model=google/gemma-3-1b-it
BASE: Hello to you too! How's your day going so far? 😊 

Is there anything you'd like to chat about, or anything I can


In [17]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

# FT only (Colab won't reliably run both)
make stop >/dev/null 2>&1 || true
make start-ft
sleep 15

python scripts/smoke_test.py --mode ft


bash scripts/start_ft_vllm.sh
Starting FT vLLM
HOST=0.0.0.0
PORT=8001
BASE_MODEL_ID=google/gemma-3-1b-it
ADAPTER_PATH=/content/drive/MyDrive/slm-hosting-playbook-artifacts/live_backup/gemma-3-1b-it-lora-20260114-034307
Expected endpoints:
http://0.0.0.0:8001/v1/models
http://0.0.0.0:8001/v1/chat/completions
Started FT vLLM (pid 4731)
Logs: /content/slm-hosting-playbook/gemma-slm-hosting/runs/logs/ft.log
MODE=ft
models_url=http://127.0.0.1:8001/v1/models
chat_url=http://127.0.0.1:8001/v1/chat/completions
chat_method=POST
chat_model=ft
FT: Hello! How can I help you today? 😊


In [18]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

mkdir -p runs/perf

python bench/perf.py \
  --url http://127.0.0.1:8001/v1 \
  --prompts data/perf_prompts.jsonl \
  --out runs/perf/perf_ft.json

ls -lh runs/perf/perf_ft.json


url=http://127.0.0.1:8001
model=ft
concurrency=1
total_requests=50
success_count=50
error_count=0
latency_ms_p50=449.83
latency_ms_p95=785.02
throughput_rps=2.05
-rw-r--r-- 1 root root 1.5K Jan 18 03:17 runs/perf/perf_ft.json


In [19]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

make snapshot
ls -lt runs/diag | head


bash scripts/diag_snapshot.sh
Snapshot: /content/slm-hosting-playbook/gemma-slm-hosting/runs/diag/20260118-031750
total 20
-rw-r--r-- 1 root root 16084 Jan 18 03:17 20260118-031750.tar.gz
drwxr-xr-x 2 root root  4096 Jan 18 03:17 20260118-031750


In [20]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

# Ensure expected log locations exist for snapshot script
mkdir -p runs
if [ -f runs/logs/base.log ]; then cp -f runs/logs/base.log runs/base.log; fi
if [ -f runs/logs/ft.log ]; then cp -f runs/logs/ft.log runs/ft.log; fi

make snapshot
ls -lt runs/diag | head


bash scripts/diag_snapshot.sh
Snapshot: /content/slm-hosting-playbook/gemma-slm-hosting/runs/diag/20260118-031801
total 40
-rw-r--r-- 1 root root 16087 Jan 18 03:18 20260118-031801.tar.gz
drwxr-xr-x 2 root root  4096 Jan 18 03:18 20260118-031801
-rw-r--r-- 1 root root 16084 Jan 18 03:17 20260118-031750.tar.gz
drwxr-xr-x 2 root root  4096 Jan 18 03:17 20260118-031750


In [21]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

latest="$(ls -td runs/diag/20* | head -n 1)"
echo "Latest snapshot: $latest"
ls -la "$latest" | grep -E 'base\.log|ft\.log|runs_logs' || true


Latest snapshot: runs/diag/20260118-031801.tar.gz


In [22]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

echo "== runs/logs =="
ls -la runs/logs 2>/dev/null || true

echo
echo "== runs/*.log =="
ls -la runs/*.log 2>/dev/null || true

echo
echo "== grep for base/ft logs anywhere under runs =="
find runs -maxdepth 3 -type f -name "*.log" -print 2>/dev/null || true


== runs/logs ==
total 64
drwxr-xr-x 2 root root  4096 Jan 18 03:08 .
drwxr-xr-x 7 root root  4096 Jan 18 03:18 ..
-rw-r--r-- 1 root root 19076 Jan 18 03:15 base.log
-rw-r--r-- 1 root root 29543 Jan 18 03:17 ft.log

== runs/*.log ==
-rw-r--r-- 1 root root 19076 Jan 18 03:18 runs/base.log
-rw-r--r-- 1 root root 29543 Jan 18 03:18 runs/ft.log

== grep for base/ft logs anywhere under runs ==
runs/base.log
runs/logs/base.log
runs/logs/ft.log
runs/ft.log


In [23]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

make snapshot
ls -lt runs/diag | head


bash scripts/diag_snapshot.sh
Snapshot: /content/slm-hosting-playbook/gemma-slm-hosting/runs/diag/20260118-031857
total 60
-rw-r--r-- 1 root root 16093 Jan 18 03:18 20260118-031857.tar.gz
drwxr-xr-x 2 root root  4096 Jan 18 03:18 20260118-031857
-rw-r--r-- 1 root root 16087 Jan 18 03:18 20260118-031801.tar.gz
drwxr-xr-x 2 root root  4096 Jan 18 03:18 20260118-031801
-rw-r--r-- 1 root root 16084 Jan 18 03:17 20260118-031750.tar.gz
drwxr-xr-x 2 root root  4096 Jan 18 03:17 20260118-031750


In [26]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

PROMPT='Explain LoRA in one sentence, then give one example use-case.'
mkdir -p runs/ab

wait_ready () {
  url="$1"   # e.g. http://127.0.0.1:8000
  name="$2"  # BASE/FT
  for i in $(seq 1 120); do
    code="$(curl -sS --max-time 2 -o /dev/null -w "%{http_code}" "${url}/v1/models" || true)"
    if [ "$code" = "200" ]; then
      echo "✅ ${name} ready"
      return 0
    fi
    echo "[$i/120] waiting for ${name}... http_code=${code}"
    sleep 2
  done
  echo "❌ ${name} not ready in time"
  return 1
}

post_chat () {
  url="$1"
  model="$2"
  out="$3"
  curl -sS "${url}/v1/chat/completions" \
    -H "Content-Type: application/json" \
    -d "{\"model\":\"${model}\",\"messages\":[{\"role\":\"user\",\"content\":\"${PROMPT}\"}],\"max_tokens\":80,\"temperature\":0}" \
    > "${out}"
}

# ---------- BASE ----------
make stop >/dev/null 2>&1 || true
make start-base
wait_ready "http://127.0.0.1:8000" "BASE"
post_chat "http://127.0.0.1:8000" "google/gemma-3-1b-it" "runs/ab/base.json"
make stop >/dev/null 2>&1 || true

# ---------- FT ----------
make start-ft
wait_ready "http://127.0.0.1:8001" "FT"
post_chat "http://127.0.0.1:8001" "ft" "runs/ab/ft.json"

echo "Saved:"
ls -lh runs/ab/*.json


bash scripts/start_base_vllm.sh
Expected endpoints:
http://0.0.0.0:8000/v1/models
http://0.0.0.0:8000/v1/chat/completions
Started base vLLM (pid 7291)
Logs: /content/slm-hosting-playbook/gemma-slm-hosting/runs/logs/base.log
[1/120] waiting for BASE... http_code=000
[2/120] waiting for BASE... http_code=000
[3/120] waiting for BASE... http_code=000
[4/120] waiting for BASE... http_code=000
[5/120] waiting for BASE... http_code=000
[6/120] waiting for BASE... http_code=000
[7/120] waiting for BASE... http_code=000
[8/120] waiting for BASE... http_code=000
[9/120] waiting for BASE... http_code=000
[10/120] waiting for BASE... http_code=000
[11/120] waiting for BASE... http_code=000
[12/120] waiting for BASE... http_code=000
[13/120] waiting for BASE... http_code=000
[14/120] waiting for BASE... http_code=000
[15/120] waiting for BASE... http_code=000
[16/120] waiting for BASE... http_code=000
[17/120] waiting for BASE... http_code=000
[18/120] waiting for BASE... http_code=000
[19/120] wa

curl: (7) Failed to connect to 127.0.0.1 port 8000 after 0 ms: Connection refused
curl: (7) Failed to connect to 127.0.0.1 port 8000 after 0 ms: Connection refused
curl: (7) Failed to connect to 127.0.0.1 port 8000 after 0 ms: Connection refused
curl: (7) Failed to connect to 127.0.0.1 port 8000 after 0 ms: Connection refused
curl: (7) Failed to connect to 127.0.0.1 port 8000 after 0 ms: Connection refused
curl: (7) Failed to connect to 127.0.0.1 port 8000 after 0 ms: Connection refused
curl: (7) Failed to connect to 127.0.0.1 port 8000 after 0 ms: Connection refused
curl: (7) Failed to connect to 127.0.0.1 port 8000 after 0 ms: Connection refused
curl: (7) Failed to connect to 127.0.0.1 port 8000 after 0 ms: Connection refused
curl: (7) Failed to connect to 127.0.0.1 port 8000 after 0 ms: Connection refused
curl: (7) Failed to connect to 127.0.0.1 port 8000 after 0 ms: Connection refused
curl: (7) Failed to connect to 127.0.0.1 port 8000 after 0 ms: Connection refused
curl: (7) Failed

In [27]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

make stop >/dev/null 2>&1 || true
pkill -f "vllm.entrypoints.openai.api_server" 2>/dev/null || true
pkill -f "scripts/healthcheck" 2>/dev/null || true
pkill -f "curl .*8000" 2>/dev/null || true
pkill -f "curl .*8001" 2>/dev/null || true

echo "Remaining vLLM processes:"
ps aux | egrep 'vllm|api_server' | grep -v grep || true


Remaining vLLM processes:
root        6448  6.3  2.1 13258584 1918980 ?    Sl   03:19   0:20 python3 -m vllm.entrypoints.openai.api_server --host 0.0.0.0 --port 8000 --model google/gemma-3-1b-it --gpu-memory-utilization 0.90 --max-model-len 2048 --max-num-seqs 128 --tensor-parallel-size 1 --dtype auto


In [28]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

echo "== BASE output =="
python - <<'PY'
import json
d=json.load(open("runs/ab/base.json"))
print(d["model"])
print(d["choices"][0]["message"]["content"])
PY

echo
echo "== FT output =="
python - <<'PY'
import json
d=json.load(open("runs/ab/ft.json"))
print(d["model"])
print(d["choices"][0]["message"]["content"])
PY


== BASE output ==
google/gemma-3-1b-it
LoRA (Low-Rank Adaptation) is a technique that efficiently fine-tunes large language models by only training a small number of additional parameters, significantly reducing computational costs and memory requirements.

**Example Use-Case:** Adapting a large language model to generate creative writing prompts for a specific genre.

== FT output ==
ft
LoRA stands for Low-Rank Adaptation, a technique used to fine-tune large language models (LLMs) with limited computational resources.

One example use-case is fine-tuning a pre-trained LLM for a specific task, such as sentiment analysis or question answering, using only a small subset of the original model's parameters. This can significantly reduce the computational cost and time required


In [29]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

mkdir -p runs/blog_bundle
cp -f runs/ab/base.json runs/ab/ft.json runs/blog_bundle/

# include perf if you already ran it
[ -f runs/perf/perf_ft.json ] && cp -f runs/perf/perf_ft.json runs/blog_bundle/ || true
[ -f runs/perf/perf_base.json ] && cp -f runs/perf/perf_base.json runs/blog_bundle/ || true

# include logs if present
[ -f runs/logs/base.log ] && cp -f runs/logs/base.log runs/blog_bundle/ || true
[ -f runs/logs/ft.log ] && cp -f runs/logs/ft.log runs/blog_bundle/ || true

ls -lh runs/blog_bundle


total 56K
-rw-r--r-- 1 root root  932 Jan 18 03:25 base.json
-rw-r--r-- 1 root root  18K Jan 18 03:25 base.log
-rw-r--r-- 1 root root  979 Jan 18 03:25 ft.json
-rw-r--r-- 1 root root  24K Jan 18 03:25 ft.log
-rw-r--r-- 1 root root 1.5K Jan 18 03:25 perf_ft.json


In [30]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

# zip it (easy to download)
cd runs
zip -r blog_bundle.zip blog_bundle >/dev/null
ls -lh blog_bundle.zip


-rw-r--r-- 1 root root 12K Jan 18 03:26 blog_bundle.zip


In [31]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

mkdir -p artifacts/blog_bundle
cp -a runs/blog_bundle/. artifacts/blog_bundle/
ls -lh artifacts/blog_bundle


total 56K
-rw-r--r-- 1 root root  932 Jan 18 03:25 base.json
-rw-r--r-- 1 root root  18K Jan 18 03:25 base.log
-rw-r--r-- 1 root root  979 Jan 18 03:25 ft.json
-rw-r--r-- 1 root root  24K Jan 18 03:25 ft.log
-rw-r--r-- 1 root root 1.5K Jan 18 03:25 perf_ft.json


In [32]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

NEW_DIR="artifacts/poc_run"

mkdir -p artifacts
rm -rf "$NEW_DIR"
mv artifacts/blog_bundle "$NEW_DIR"

echo "Renamed to: $NEW_DIR"
find "$NEW_DIR" -maxdepth 1 -type f -printf "%f\n" | sort


Renamed to: artifacts/poc_run
base.json
base.log
ft.json
ft.log
perf_ft.json


In [33]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

cat > artifacts/poc_run/README.md <<'EOF'
PoC run evidence (Colab)

Files:
- base.json, ft.json: A/B prompt responses (BASE vs LoRA FT) from vLLM OpenAI-compatible endpoints.
- perf_ft.json: Simple perf numbers captured via bench/perf.py.
- base.log, ft.log: vLLM server logs for the run.
EOF

sed -n '1,120p' artifacts/poc_run/README.md


PoC run evidence (Colab)

Files:
- base.json, ft.json: A/B prompt responses (BASE vs LoRA FT) from vLLM OpenAI-compatible endpoints.
- perf_ft.json: Simple perf numbers captured via bench/perf.py.
- base.log, ft.log: vLLM server logs for the run.


In [34]:
%%bash
set -euo pipefail
cd /content/slm-hosting-playbook/gemma-slm-hosting

zip -r artifacts_poc_run.zip artifacts/poc_run
ls -lh artifacts_poc_run.zip


  adding: artifacts/poc_run/ (stored 0%)
  adding: artifacts/poc_run/README.md (deflated 25%)
  adding: artifacts/poc_run/base.log (deflated 81%)
  adding: artifacts/poc_run/ft.json (deflated 45%)
  adding: artifacts/poc_run/perf_ft.json (deflated 54%)
  adding: artifacts/poc_run/ft.log (deflated 77%)
  adding: artifacts/poc_run/base.json (deflated 45%)
-rw-r--r-- 1 root root 12K Jan 18 03:32 artifacts_poc_run.zip
